# Model creation on San Francisco Crime Data

To give an introduction this is the second notebook on the model deployment process that I will try to achieve . We initially did some data exploration to remove outliers and used Vaex library to process data and save the processed data for further usage. <a href="https://github.com/ankur26/SFCrimePrediction/blob/master/notebooks/01_data_exploration_and_feature_engineering.ipynb">Link to first notebook </a>

This notebook will entail the model creation process and we will find the best fitting model



In [48]:
import pandas as pd
import numpy as np

In [49]:
train_data = pd.read_csv("/root/SFCrimePrediction/data/processed/train_processed.csv")
test_data = pd.read_csv("/root/SFCrimePrediction/data/processed/test_processed.csv")

In [50]:
train_data.drop('Unnamed: 0',axis=1,inplace=True)

In [51]:
test_data.drop('Unnamed: 0',axis=1,inplace=True)

In [52]:
train_features = train_data.drop("Category",axis=1).fillna("N/A")
train_target = train_data["Category"]

In [53]:
features = train_features.dtypes.reset_index()

In [54]:
cat_features = features.loc[features[0] == "object","index"].values

In [55]:
from catboost import *

In [64]:
params = {}
params["loss_function"] = "MultiClass"
params["iterations"] = 1000
params["learning_rate"] = 0.01
params["task_type"] = "GPU"
params["depth"] = 10
# params["save_snapshot"] = True
# params["snapshot_interval"] = 1200


In [65]:
cv_data  = cv(
            params=params,
            pool = Pool(data=train_features,label=train_target,cat_features=cat_features),
            shuffle=True,
            fold_count=5,
            stratified=True,
            verbose=True)


0:	learn: 3.6220125	test: 3.6220542	best: 3.6220542 (0)
1:	learn: 3.5847107	test: 3.5848357	best: 3.5848357 (1)
2:	learn: 3.5501715	test: 3.5503797	best: 3.5503797 (2)
3:	learn: 3.5181159	test: 3.5184678	best: 3.5184678 (3)
4:	learn: 3.4881495	test: 3.4885744	best: 3.4885744 (4)
5:	learn: 3.4599101	test: 3.4603781	best: 3.4603781 (5)
6:	learn: 3.4331096	test: 3.4336424	best: 3.4336424 (6)
7:	learn: 3.4076835	test: 3.4082729	best: 3.4082729 (7)
8:	learn: 3.3836078	test: 3.3842538	best: 3.3842538 (8)
9:	learn: 3.3607072	test: 3.3614370	best: 3.3614370 (9)
10:	learn: 3.3388498	test: 3.3396288	best: 3.3396288 (10)
11:	learn: 3.3180006	test: 3.3188292	best: 3.3188292 (11)
12:	learn: 3.2979138	test: 3.2988096	best: 3.2988096 (12)
13:	learn: 3.2786070	test: 3.2795200	best: 3.2795200 (13)
14:	learn: 3.2600328	test: 3.2609845	best: 3.2609845 (14)
15:	learn: 3.2410242	test: 3.2420557	best: 3.2420557 (15)
16:	learn: 3.2237599	test: 3.2248449	best: 3.2248449 (16)
17:	learn: 3.2068178	test: 3.20793

231:	learn: 2.3958784	test: 2.4067739	best: 2.4067739 (231)
232:	learn: 2.3951310	test: 2.4060772	best: 2.4060772 (232)
233:	learn: 2.3944619	test: 2.4054535	best: 2.4054535 (233)
234:	learn: 2.3937485	test: 2.4047786	best: 2.4047786 (234)
235:	learn: 2.3930842	test: 2.4041336	best: 2.4041336 (235)
236:	learn: 2.3924111	test: 2.4034970	best: 2.4034970 (236)
237:	learn: 2.3916792	test: 2.4028342	best: 2.4028342 (237)
238:	learn: 2.3910326	test: 2.4022304	best: 2.4022304 (238)
239:	learn: 2.3902395	test: 2.4014850	best: 2.4014850 (239)
240:	learn: 2.3895616	test: 2.4008560	best: 2.4008560 (240)
241:	learn: 2.3889090	test: 2.4002463	best: 2.4002463 (241)
242:	learn: 2.3882762	test: 2.3996578	best: 2.3996578 (242)
243:	learn: 2.3876248	test: 2.3990550	best: 2.3990550 (243)
244:	learn: 2.3869938	test: 2.3984692	best: 2.3984692 (244)
245:	learn: 2.3863749	test: 2.3978762	best: 2.3978762 (245)
246:	learn: 2.3857734	test: 2.3973105	best: 2.3973105 (246)
247:	learn: 2.3851372	test: 2.3967117	be

382:	learn: 2.3310244	test: 2.3492335	best: 2.3492335 (382)
383:	learn: 2.3307406	test: 2.3489993	best: 2.3489993 (383)
384:	learn: 2.3304989	test: 2.3488052	best: 2.3488052 (384)
385:	learn: 2.3301804	test: 2.3485492	best: 2.3485492 (385)
386:	learn: 2.3298699	test: 2.3482973	best: 2.3482973 (386)
387:	learn: 2.3296177	test: 2.3480700	best: 2.3480700 (387)
388:	learn: 2.3292790	test: 2.3477871	best: 2.3477871 (388)
389:	learn: 2.3290474	test: 2.3475931	best: 2.3475931 (389)
390:	learn: 2.3287413	test: 2.3473477	best: 2.3473477 (390)
391:	learn: 2.3284201	test: 2.3470811	best: 2.3470811 (391)
392:	learn: 2.3281155	test: 2.3468395	best: 2.3468395 (392)
393:	learn: 2.3278760	test: 2.3466527	best: 2.3466527 (393)
394:	learn: 2.3275988	test: 2.3464235	best: 2.3464235 (394)
395:	learn: 2.3273386	test: 2.3462243	best: 2.3462243 (395)
396:	learn: 2.3270639	test: 2.3459986	best: 2.3459986 (396)
397:	learn: 2.3268536	test: 2.3458252	best: 2.3458252 (397)
398:	learn: 2.3265795	test: 2.3455961	be

551:	learn: 2.2959787	test: 2.3230955	best: 2.3230955 (551)
552:	learn: 2.2958036	test: 2.3229581	best: 2.3229581 (552)
553:	learn: 2.2956545	test: 2.3228693	best: 2.3228693 (553)
554:	learn: 2.2954913	test: 2.3227516	best: 2.3227516 (554)
555:	learn: 2.2953486	test: 2.3226711	best: 2.3226711 (555)
556:	learn: 2.2951491	test: 2.3225379	best: 2.3225379 (556)
557:	learn: 2.2949787	test: 2.3224243	best: 2.3224243 (557)
558:	learn: 2.2948503	test: 2.3223361	best: 2.3223361 (558)
559:	learn: 2.2946791	test: 2.3222284	best: 2.3222284 (559)
560:	learn: 2.2945055	test: 2.3221125	best: 2.3221125 (560)
561:	learn: 2.2943211	test: 2.3219828	best: 2.3219828 (561)
562:	learn: 2.2941736	test: 2.3218919	best: 2.3218919 (562)
563:	learn: 2.2939764	test: 2.3217495	best: 2.3217495 (563)
564:	learn: 2.2938000	test: 2.3216430	best: 2.3216430 (564)
565:	learn: 2.2936289	test: 2.3215250	best: 2.3215250 (565)
566:	learn: 2.2934947	test: 2.3214112	best: 2.3214112 (566)
567:	learn: 2.2933487	test: 2.3213053	be

732:	learn: 2.2703071	test: 2.3068242	best: 2.3068242 (732)
733:	learn: 2.2701637	test: 2.3067371	best: 2.3067371 (733)
734:	learn: 2.2700539	test: 2.3066787	best: 2.3066787 (734)
735:	learn: 2.2699352	test: 2.3066184	best: 2.3066184 (735)
736:	learn: 2.2698332	test: 2.3065692	best: 2.3065692 (736)
737:	learn: 2.2697104	test: 2.3064939	best: 2.3064939 (737)
738:	learn: 2.2695605	test: 2.3064052	best: 2.3064052 (738)
739:	learn: 2.2694321	test: 2.3063355	best: 2.3063355 (739)
740:	learn: 2.2692762	test: 2.3062361	best: 2.3062361 (740)
741:	learn: 2.2691610	test: 2.3061699	best: 2.3061699 (741)
742:	learn: 2.2690323	test: 2.3060974	best: 2.3060974 (742)
743:	learn: 2.2688914	test: 2.3060172	best: 2.3060172 (743)
744:	learn: 2.2687637	test: 2.3059373	best: 2.3059373 (744)
745:	learn: 2.2686411	test: 2.3058718	best: 2.3058718 (745)
746:	learn: 2.2685137	test: 2.3057839	best: 2.3057839 (746)
747:	learn: 2.2684058	test: 2.3057063	best: 2.3057063 (747)
748:	learn: 2.2682589	test: 2.3056117	be

899:	learn: 2.2507827	test: 2.2966238	best: 2.2966238 (899)
900:	learn: 2.2506647	test: 2.2965680	best: 2.2965680 (900)
901:	learn: 2.2505777	test: 2.2965251	best: 2.2965251 (901)
902:	learn: 2.2504815	test: 2.2964719	best: 2.2964719 (902)
903:	learn: 2.2503921	test: 2.2964311	best: 2.2964311 (903)
904:	learn: 2.2502741	test: 2.2963691	best: 2.2963691 (904)
905:	learn: 2.2501388	test: 2.2963056	best: 2.2963056 (905)
906:	learn: 2.2500302	test: 2.2962589	best: 2.2962589 (906)
907:	learn: 2.2498870	test: 2.2961910	best: 2.2961910 (907)
908:	learn: 2.2497817	test: 2.2961314	best: 2.2961314 (908)
909:	learn: 2.2496466	test: 2.2960634	best: 2.2960634 (909)
910:	learn: 2.2495231	test: 2.2959960	best: 2.2959960 (910)
911:	learn: 2.2494328	test: 2.2959410	best: 2.2959410 (911)
912:	learn: 2.2492969	test: 2.2958762	best: 2.2958762 (912)
913:	learn: 2.2491698	test: 2.2958093	best: 2.2958093 (913)
914:	learn: 2.2490805	test: 2.2957475	best: 2.2957475 (914)
915:	learn: 2.2489485	test: 2.2956791	be

The metric in the competition used is called as multi class logarithmic loss which basically entails the probability of the crime happening to you.
So for example we have a set of four crimes namely 1,2,3,4 and so the output for crime 1 should be a vector [1,0,0,0] as we know that the probability of crime 1 occuring is 1 and if our predicted outcome is [0.8,0.2,0.0] the model predicted that the outcome for crime 1 is 0.8 (80% percent chance of crime 1) and crime 2 is 0.2 (20% of chance of crime 2).

So the log loss will be calculated as the sum (output * log(probabilities)) which is then normalized across all samples by dividing by the sample size.

For further detail here is the link : <a href="https://www.kaggle.com/wiki/MultiClassLogLoss"> Multi Class Log Loss</a>